##setup

In [ ]:
%%capture
!pip install compressai
# !pip install -i https://test.pypi.org/simple/ imgstitch

NotImplementedError: ignored

In [ ]:
import argparse
import random
import shutil
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math

from torch.utils.data import DataLoader
from torchvision import transforms

# from compressai.datasets import ImageFolder
from compressai.losses import RateDistortionLoss
from compressai.optimizers import net_aux_optimizer
from compressai.zoo import image_models
from compressai.datasets import ImageFolder

class AverageMeter:
    """Compute running average."""

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class CustomDataParallel(nn.DataParallel):
    """Custom DataParallel to access the module methods."""

    def __getattr__(self, key):
        try:
            return super().__getattr__(key)
        except AttributeError:
            return getattr(self.module, key)


def configure_optimizers(net, args):
    """Separate parameters for the main optimizer and the auxiliary optimizer.
    Return two optimizers"""
    conf = {
        "net": {"type": "Adam", "lr": args.learning_rate},
        "aux": {"type": "Adam", "lr": args.aux_learning_rate},
    }
    optimizer = net_aux_optimizer(net, conf)
    return optimizer["net"], optimizer["aux"]



In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
import cv2

def ozlem(ori_image):

  image = ori_image[0].requires_grad_(requires_grad=True)
  image = image.detach().cpu().numpy().transpose(1,2,0)
  # image = image.detach().cpu().numpy()
  # print(image.shape)

  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # print(gray_image.shape)

  orb = cv2.ORB_create(nfeatures=2000)

  # breakpoint()
  kp, des = orb.detectAndCompute(image, None)

  kp_image = cv2.drawKeypoints(np.uint8(image), kp, None, color=(0, 255, 0)).astype(np.float32)

  kp_image = torch.from_numpy(np.asarray(kp_image)).to('cuda').requires_grad_(requires_grad=True)
  # print(kp_image.size())

  return kp_image

In [ ]:
img = cv2.imread('/content/MyDrive/MyDrive/inference_time/data/Kodak/train/1.png')
img = torch.from_numpy(img)

##code

In [ ]:
a = np.zeros((2,3))
a = torch.from_numpy(a)

In [ ]:
print(a.size())
print(a.unsqueeze(0).size())

torch.Size([2, 3])
torch.Size([1, 2, 3])


In [ ]:
def train_one_epoch(
    model, criterion, train_dataloader, optimizer, aux_optimizer, epoch, clip_max_norm
):
    model.train()
    device = next(model.parameters()).device

    for i, d in enumerate(train_dataloader):

        image = d.to(device)
        # target = d[1].to(device).to(torch.float32)
        target = image
        optimizer.zero_grad()
        aux_optimizer.zero_grad()

        out_net = model(image)
        # print(ozlem(out_net['x_hat']).shape)
        # print(ozlem(target).shape)


        out_criterion_1 = criterion(out_net['x_hat'], target)
        out_criterion_2 = criterion(ozlem(out_net['x_hat']).unsqueeze(0), ozlem(target).unsqueeze(0))

        out_criterion_2["loss"].backward()
        if clip_max_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_max_norm)
        optimizer.step()

        aux_loss = model.aux_loss()
        aux_loss.backward()
        aux_optimizer.step()

        if i % 4 == 0:
            print(
                f"Train epoch {epoch}: ["
                f"{i*len(d)}/{len(train_dataloader.dataset)}"
                f" ({100. * i / len(train_dataloader):.0f}%)]"
                f'\tMSE loss: {out_criterion_1["loss"].item():.3f} |'
                f'\tMSE loss: {out_criterion_2["loss"].item():.3f} |'
                f"\tAux loss: {aux_loss.item():.2f}"
            )

    # return out_criterion_2["mse_loss"].item(), out_criterion["bpp_loss"].item(), out_criterion["loss"].item()


def test_epoch(epoch, test_dataloader, model, criterion):
    model.eval()
    device = next(model.parameters()).device

    loss = AverageMeter()
    bpp_loss = AverageMeter()
    mse_loss = AverageMeter()
    aux_loss = AverageMeter()

    with torch.no_grad():
        for d in test_dataloader:
            image = d[0].to(device)
            target = d[1].to(device)
            out_net = model(image)
            out_criterion = criterion(out_net, target)

            aux_loss.update(model.aux_loss())
            bpp_loss.update(out_criterion["bpp_loss"])
            loss.update(out_criterion["loss"])
            mse_loss.update(out_criterion["mse_loss"])

    print(
        f"Test epoch {epoch}: Average losses:"
        f"\tLoss: {loss.avg:.3f} |"
        f"\tMSE loss: {mse_loss.avg:.3f} |"
        f"\tBpp loss: {bpp_loss.avg:.2f} |"
        f"\tAux loss: {aux_loss.avg:.2f}\n"
    )

    return loss.avg


def save_checkpoint(state, is_best, filename="checkpoint.pth.tar"):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, "checkpoint_best_loss.pth.tar")

In [ ]:
class RateDistortionLoss(nn.Module):
    """Custom rate distortion loss with a Lagrangian parameter."""

    def __init__(self, lmbda=0.01, metric="mse", return_type="all"):
        super().__init__()

        self.metric = nn.MSELoss()

        self.lmbda = lmbda
        self.return_type = return_type

    def forward(self, output, target):
        N, _, H, W = target.size()
        out = {}
        num_pixels = N * H * W

        # out["bpp_loss"] = sum(
        #     (torch.log(likelihoods).sum() / (-math.log(2) * num_pixels))
        #     for likelihoods in output["likelihoods"].values()
        # )

        # out["mse_loss"] = self.metric(output["x_hat"], target)
        out["loss"] = self.metric(output, target)* 255**2
        
        # out["loss"] = self.lmbda * distortion + out["bpp_loss"]
        return out

##main

In [ ]:
def main():

    class arguments:
      def __init__(self):
        self.model = 'bmshj2018-hyperprior'
        self.num_workers = 1
        self.dataset = '/content/MyDrive/MyDrive/inference_time/data/Kodak'
        self.batch_size = 1
        self.test_batch_size = 2
        self.cuda = True
        self.epochs = 50
        self.patch_size = 64
        self.learning_rate = 0.001
        self.aux_learning_rate = 0.001
        self.lmbda = 0.001
        self.lmbda2 = 1
        self.save = True
        self.seed = False
        self.clip_max_norm = 255.0
        self.checkpoint = False

    args = arguments()

    if args.seed is not None:
        torch.manual_seed(args.seed)
        random.seed(args.seed)

    train_transforms = transforms.Compose(
        [transforms.ToTensor()]
    )

    test_transforms = transforms.Compose(
        [transforms.ToTensor()]
    )

    train_dataset = ImageFolder(args.dataset, split="train", transform=train_transforms)
    # test_dataset = ImageFolder(args.dataset, split="", transform=test_transforms)

    device = "cuda" if args.cuda and torch.cuda.is_available() else "cpu"

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        shuffle=True,
        pin_memory=(device == "cuda"),
    )

    # test_dataloader = DataLoader(
    #     test_dataset,
    #     batch_size=args.test_batch_size,
    #     num_workers=args.num_workers,
    #     shuffle=False,
    #     pin_memory=(device == "cuda"),
    # )

    net = image_models[args.model](quality=3)
    ############## to give 5 different images:
    # net.g_a[0] = torch.nn.Conv2d(15, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) 
    ##########################################
    net = net.to(device)

    if args.cuda and torch.cuda.device_count() > 1:
        net = CustomDataParallel(net)

    optimizer, aux_optimizer = configure_optimizers(net, args)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
    criterion = RateDistortionLoss(lmbda=args.lmbda)

    last_epoch = 0
    if args.checkpoint:  # load from previous checkpoint
        print("Loading", args.checkpoint)
        checkpoint = torch.load(args.checkpoint, map_location=device)
        last_epoch = checkpoint["epoch"] + 1
        net.load_state_dict(checkpoint["state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        aux_optimizer.load_state_dict(checkpoint["aux_optimizer"])
        lr_scheduler.load_state_dict(checkpoint["lr_scheduler"])

    best_loss = float("inf")
    mse_loss = np.zeros((args.epochs,1))
    bpp_loss = np.zeros((args.epochs,1))
    loss = np.zeros((args.epochs,1))
    for epoch in range(last_epoch, args.epochs):
        if epoch%5==0: optimizer.param_groups[0]['lr'] /= 2
        print(f"\nLearning rate: {optimizer.param_groups[0]['lr']}")
        # mse_loss[epoch], bpp_loss[epoch], loss[epoch] = train_one_epoch(
        train_one_epoch(
            net,
            criterion,
            train_dataloader,
            optimizer,
            aux_optimizer,
            epoch,
            args.clip_max_norm,
        )
        # loss = test_epoch(epoch, test_dataloader, net, criterion)
        # lr_scheduler.step(loss)

        # is_best = loss < best_loss
        # best_loss = min(loss, best_loss)

        if args.save:
            save_checkpoint(
                {
                    "epoch": epoch,
                    "state_dict": net.state_dict(),
                    # "loss": loss,
                    "optimizer": optimizer.state_dict(),
                    "aux_optimizer": aux_optimizer.state_dict(),
                    "lr_scheduler": lr_scheduler.state_dict(),
                },
                is_best=0,
            )
    return mse_loss, bpp_loss, loss

##train

In [ ]:
main()


Learning rate: 0.0005
Train epoch 0: [0/24 (0%)]	MSE loss: 0.631 |	MSE loss: 1858.163 |	Aux loss: 5277.59
Train epoch 0: [4/24 (17%)]	MSE loss: 0.124 |	MSE loss: 0.007 |	Aux loss: 5277.44
Train epoch 0: [8/24 (33%)]	MSE loss: 0.169 |	MSE loss: 0.006 |	Aux loss: 5277.28
Train epoch 0: [12/24 (50%)]	MSE loss: 0.461 |	MSE loss: 0.018 |	Aux loss: 5277.13
Train epoch 0: [16/24 (67%)]	MSE loss: 0.273 |	MSE loss: 0.004 |	Aux loss: 5276.98
Train epoch 0: [20/24 (83%)]	MSE loss: 0.324 |	MSE loss: 0.052 |	Aux loss: 5276.82

Learning rate: 0.0005
Train epoch 1: [0/24 (0%)]	MSE loss: 0.244 |	MSE loss: 0.020 |	Aux loss: 5276.67
Train epoch 1: [4/24 (17%)]	MSE loss: 0.199 |	MSE loss: 0.001 |	Aux loss: 5276.52
Train epoch 1: [8/24 (33%)]	MSE loss: 0.225 |	MSE loss: 0.002 |	Aux loss: 5276.36
Train epoch 1: [12/24 (50%)]	MSE loss: 0.188 |	MSE loss: 0.001 |	Aux loss: 5276.21
Train epoch 1: [16/24 (67%)]	MSE loss: 0.240 |	MSE loss: 0.001 |	Aux loss: 5276.06
Train epoch 1: [20/24 (83%)]	MSE loss: 0.157 |